In [ ]:
import sys
!{sys.executable} pip -m install sklearn


In [128]:
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.linear_model import SGDClassifier

import gensim
from gensim.parsing.porter import PorterStemmer
import json
import csv

In [95]:
TEXTFILE = "AllText.json"
FileJson = json.load(open(TEXTFILE))
RawSentences = {}
PortStem = PorterStemmer()
for i, sentenceid in enumerate(FileJson):
    RawSentences[int(sentenceid)] = PortStem.stem_documents(gensim.utils.simple_preprocess(FileJson[sentenceid]))
pprint(list(RawSentences.values())[:5])

[['ve',
  'said',
  'it',
  'befor',
  'and',
  'will',
  'sai',
  'it',
  'again',
  'the',
  'mainstream',
  'media',
  'is',
  'out',
  'to',
  'bring',
  'down',
  'my',
  'administr',
  'need',
  'you',
  'to',
  'take',
  'the',
  'media',
  'account',
  'survei',
  'to',
  'do',
  'your',
  'part',
  'to',
  'fight',
  'back',
  'against',
  'the',
  'fake',
  'new',
  'attack',
  'and',
  'decept'],
 ['if',
  'we',
  'want',
  'to',
  'chang',
  'congress',
  'we',
  'need',
  'to',
  'chang',
  'who',
  'we',
  'send',
  'to',
  'congress'],
 ['happi',
  'th',
  'of',
  'juli',
  'let',
  'celebr',
  'our',
  'freedom',
  'and',
  'independ',
  'todai',
  'and',
  'cherish',
  'them',
  'alwai',
  'freedom',
  'is',
  'never',
  'more',
  'than',
  'on',
  'gener',
  'awai',
  'from',
  'extinct',
  'we',
  'didn',
  'pass',
  'it',
  'to',
  'our',
  'children',
  'in',
  'the',
  'bloodstream',
  'it',
  'must',
  'be',
  'fought',
  'for',
  'protect',
  'and',
  'hand',
  

In [97]:
Labels = {}
LABELSFILE = "Text.csv"
LabeledData = {} 
with open(LABELSFILE, 'r') as f:
    File = csv.DictReader(f, delimiter=',')
    for row in File:
        ID = int(row['ID'])        
        if row['ClearMajority'] != '-':
            Labels[ID] = int(row['ClearMajority'])
        elif row['SoftMajority'] != '-':
            Labels[ID] = int(row['SoftMajority'])
        elif row['damon'] != '-':
            Labels[ID] = int(row['damon'])
        LabeledData[ID] = RawSentences[ID]
IDs = [ID for ID in Labels]
print(IDs[:5])
Corpus = [' '.join(LabeledData[ID]) for ID in IDs]
print(Corpus[:5])
Labels = [Labels[ID] for ID in IDs]
print(Labels[:5])

[140707070135296, 1468073206631424, 1973034292771842, 1826638557631491, 187993745381382]
['work with presid trump on smart reform that reduc crime and cut cost', 'sign if you think sex ed should be sex posit', 'tommi gregori for state hous district', 'need help get on track for retir call me at to learn more about the option woodmenlif ha to help you', 'yike stori show trump is true pig']
[2, 1, 0, 1, -1]


In [134]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', 
                        encoding='latin-1', ngram_range=(1, 2), stop_words='english')

X_train = tfidf.fit_transform(Corpus)
clf = MultinomialNB()

SGD = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None)

scores = cross_val_score(SGD, X_train, Labels, cv = 10)
print(sum(scores)/10)

#SGD = GridSearchCV(svm.SVC(), tuned_parameters, cv=5,
                       #scoring='%s_macro' % score)

#SGD.fit(X_train, Labels)
#     print(SGD.best_params_)
#     means = SGD.cv_results_['mean_test_score']
#     stds = SGD.cv_results_['std_test_score']

#     print(means)
    
#     for mean, std, params in zip(means, stds, SGD.cv_results_['params']):
#         print("%0.3f (+/-%0.03f) for %r"
#               % (mean, std * 2, params))


0.4667315233785822
